In [8]:
import numpy as np
from pandas import read_csv
from pandas import to_datetime
import math

# load the train data
dataframe = read_csv('dataset/train.csv.gz',usecols=[i for i in range(0,211)], engine='python', header=0)
trainData = dataframe.values
trainDataOriginal = np.nan_to_num(trainData)
trainData = trainDataOriginal
# load the test data
dataframe = read_csv('dataset/test_2.csv.gz',usecols=[i for i in range(0,146)], engine='python', header=0)
testData = dataframe.values
testData = np.nan_to_num(testData)
testX = testData

#Separate the train data to input and output
trainX = trainData[0:len(trainData),0:146]
trainY = trainData[0:len(trainData),147:211]

In [9]:
#parameters
batch_size = 64
input_dim  = 146
output_dim = 64

In [10]:
from keras.activations import linear
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import adam
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

class Model:
    
    _scalerInput = MinMaxScaler(feature_range=(0, 1))
    _scalerOutput = MinMaxScaler(feature_range=(0, 1))
    
    def __init__(self):        
        #self.optimizer = adam(0.001, 0.9, 0.999, 1e-08, 0.0)
        self.model = Sequential()
        self.model.add(LSTM(output_dim, input_shape=(batch_size, input_dim), return_sequences=True))
        self.model.add(Dense(output_dim, activation='linear'))
        #Configures the model for training.
        self.model.compile(loss="mean_squared_error",optimizer="adam")
    def train(self, trainX, trainY):
        # normalize the input
        trainX = self._scalerInput.fit_transform(trainX)
        trainY = self._scalerOutput.fit_transform(trainY)
        #Put it into batches
        trainX = np.reshape(trainX,(len(trainX)/batch_size,batch_size,trainX.shape[1]))
        trainY = np.reshape(trainY,(len(trainY)/batch_size,batch_size,trainY.shape[1]))
        #Trains the model for a fixed number of epochs (iterations on a dataset).
        self.model.fit(trainX,trainY,batch_size=batch_size,epochs=1000)
    def predict(self,testX):
        # normalize the input
        testX = self._scalerInput.fit_transform(testX)
        #Put it into batches
        testX  = np.reshape(testX,(len(testX)/batch_size,batch_size,testX.shape[1])) 
        #make the prediction
        testY = self.model.predict(testX)
        #reshape, make 2D array
        testY = testY.reshape(len(testData),output_dim)
        #denormalize the result
        return self._scalerOutput.inverse_transform(testY)
    

In [11]:
#train the model
nn = Model()
nn.train(trainX, trainY)
result = nn.predict(testX)

Epoch 1/2
625/625 [==============================] - 2s - loss: 0.1510     
Epoch 2/2
625/625 [==============================] - 2s - loss: 0.0312     


In [12]:
import gzip
#create the result.cs.gzip
content = "Id,Predicted\n"
for y in range(result.shape[0]):
    for x in range(result.shape[1]-2):
       content +=str(y+1)+"_"+str(x+1)+","+str(result[y][x])+"\n"

with gzip.open('./result.csv.gz', 'wb') as f:
    f.write(content)